# Purpose

This notebook is setup to query the [OSTI.gov](https://www.OSTI.gov) API for project records. The goals for the code located herein are:

1. Determine what fields are available for different records in OSTI
2. Design a DOE Solar Energy Technologies Office (SETO) query that only pulls that technology office's data
    * **Note:** this required `{'sponsoring_org': '"EE-4S"'}` in order to work the same as the browser-based search query. It appears that the syntax of the API and of the browser-based search is not fully harmonized right now, although I'm told it will be in the future.
3. Build the query to work using an arbitrarily-large list of formatted project IDs, assuming the Solar Information Management System (SIMS) project code syntax as the input.
    * **Note:** SIMS is an internal DOE system
    * **Note 2:** This feature is not yet implemented. Currently the CSV output includes all records available in the timeframe specified by `` and ``, it does not query a specific set of projects. This can be included pretty easily as desired however.

# The Main Event

If all you want to do is extract records from OSTI's API, you needn't go any further than the next few cells. Anything past this section is simply testing code for trying to improve our queries, but isn't needed for the most basic functionality.

## What This Code Does and How to Use It

The code in the cell below does one thing only: it queries the OSTI API and saves a CSV file into the `outputs` folder that is named with today's date that contains all of the records the query produced. It would not normally be lumped into a single cell like this, given how much it's doing, but this was necessary to make its usage by third parties as simple as possible.

As the user, there are a few things you'll need to do here:

1. Set the `query_start_date` and `query_end_date` variables to specify the bounds, in time, of your query (e.g. how old and recent you want your records to be, resp.)
2. Set the `project_list` variable to something if you want to search for only specific projects
    * **Example:** if you want to find records only for the projects with DOE award numbers DE-EE0007325 and DE-EE0007326, you would write `project_list = ['EE0007325', 'EE0007326']`. Leave `project_list = []` if you don't want to search for specific projects and instead only want all projects tagged as belonging to the Solar Energy Technologies Office (SETO AKA EE-4S).
    * **WARNING:** not all records are tagged properly as belonging to a certain office. It is possible that records are tagged as being sponsored only by DOE at large or a large division within DOE (such as the Energy Efficiency and Renewable Energy, or EERE, group). The default parameters of the `osti.query_API` function will need to be changed if you want to query specific project numbers that you know were SETO-funded (so you should ignore their funding tag in OSTI for fully comprehensive results).
3. When you're done setting variables, simply hit `Shift+Enter` while still in the code block and it will execute. It should take less than a minute to complete.
4. Your output is a CSV file that will be saved in the `output` folder. Go back to the browser tab in which you clicked on the link to this notebook and click now on the output folder. You should see the file, named after today's date, in there and ready for you to download.
    * It may take another 10-15 seconds after the code has completely executed for the output file to be visible.
    * **WARNING:** when you close out of the browser tabs you opened to get here, *the file this code created will be deleted.* This is for a variety of technical reasons I won't bother you with for now, but just know that you'll need to re-run this code if you don't download that file before closing these browser tabs.

In [ ]:
#First things first, make sure we can import modifications to custom packages
import autoreload
%load_ext autoreload
%autoreload 2

import datetime as dt
import pandas as pd
import math
import os
import osti

#-----------------------------------------------------------------------------

#USER SETTINGS GO HERE

# format for dates = 'MM/DD/YYYY'
query_start_date = '01/01/2015'
query_end_date = dt.date.today().strftime('%m/%d/%Y')  # today's date

#Fill project_list with project ID strings use if you want to specify only certain 
    #projects to query
    
#FOA IDs should be formatted 'EE000XXXX' (e.g. ['EE0001234', 'EE005678'] would query
    #only for projects EE0001234 and EE005678)
    #Currently, Lab projects are not tagged with CPS ID and thus are difficult to search
    #Note that not every project is properly tagged as belonging to SETO, so you may not
        #want to use the default params dictionary and instead choose to not set a value
        #for sponsor_org
        
project_list = []

#-----------------------------------------------------------------------------

#First, need to run a single query to get the full results count
#This uses the default params which return all available SETO-only records,
    #unless you specify otherwise

_, results_count = osti.query_API(print_status = True, start_date = query_start_date,
                                 end_date=query_end_date, project_ID_list=project_list)


# Pull the full results of your query into a DataFrame
params = {'sort': 'publication_date desc', 'sponsor_org': '"EE-4S"',
          'rows': results_count}

temp, _ = osti.query_API(params=params)
df = pd.DataFrame.from_dict(temp)


#Extract the link for the landing page of each record
df['citation_link'] = df['links'].apply(osti.get_citation_URL)

#Export data to a CSV file, named using today's date
today = dt.date.today().strftime('%m-%d-%Y')
filename = today + '_OSTI_Data_Pull.csv'

df.to_csv(os.path.join(r'output/',filename))

# Cleaning and Exploring the Data

## Checking Consistency of Office Specificity

As multiple DOE program offices can be associated with a project output in the OSTI database, I want to check to make sure that the filter we're applying to get only solar-related projects is working as expected.

In [ ]:
#Convert sponsoring_org field to be a str instead of list and split on the comma delimiter to make sure 
    #EE-4S is everywhere
df['sponsor_orgs'].astype('str').str.split(", ", expand = True)[1].value_counts()

## Determining What Links We Get and Where They Go

The `links` field seems to provide some URLs for us to use, let's do some spot checks and see if they go to project landing pages or straight to the full text itself (the former is preferred over the latter).

**It looks like `'rel': 'citation'` is the link type we want to go straight to the landing page.** `'href'` key for dict gives us what we're looking for.

In [ ]:
df['citation_link'] = df['links'].apply(osti.get_citation_URL)

## What Does an Embargoed Document Look Like?

We know that certain projects can embargo their publications for different periods of time (and some, such as the SBIR program, may even *require* embargoes). As such, we want to be able to flag a record when we think it's under embargo so we can check back regularly to see when the document may finally be available to the public. So, we need to figure out if there's a pattern that we can use to generate such a flag.

**Let's first look for award EE0007326 and OSTI report ID 1490198, which I know to be currently under embargo.**

In [ ]:
params = {'sort': 'publication_date desc', 'sponsor_org': '"EE-4S"',
         'identifier': 'EE0007326'}

r = requests.get(URL, params=params)

query_date = r.headers["Date"]
results_count = r.headers['X-Total-Count']

print(f"Query was successful: {r.status_code == requests.codes.ok}")
print(f"Query made on {query_date} returned {results_count} hits")
print(f"URL used was {r.url}")


In [ ]:
df_test = pd.DataFrame.from_dict(r.json())
df_test

**Sadly, as of right now, it appears there's no good way to determine if a record is embargoed.** I thought the key may have been in the `links` column (those with only a citation-type link could be considered embargoed), but the ones lacking a full text may simply be behind a paywall of a journal, for example. So right now, I see no fool-proof way to flag these.